In [8]:
import xgboost as xgb
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

import sys

sys.path.append("../../")

import model_interpreter
from model_interpreter.interpreter import ModelInterpreter

In [2]:
model_interpreter.__version__

'1.0.0'

### Load data

In [3]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

In [4]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32



### Create catagorical variable

In [5]:
X["Over25"] = ["Over25" if a > 25 else "Under25" for a in X["HouseAge"]]

In [6]:
X[["HouseAge", "Over25"]]

,HouseAge,Over25
0,41.0,Over25
1,21.0,Under25
2,52.0,Over25
3,52.0,Over25
4,52.0,Over25
...,...,...
20635,25.0,Under25
20636,18.0,Under25
20637,17.0,Under25
20638,18.0,Under25


### Apply ohe

In [12]:
X = pd.get_dummies(X, columns=['Over25'])

In [13]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Over25_Over25,Over25_Under25
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,1,0
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,0,1
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,1,0
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,1,0
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,1,0
...,...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0,1
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0,1
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0,1
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0,1


In [14]:
X = X.drop(columns="HouseAge")

### Train model

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)
dtrain = xgb.DMatrix(data=X_train, label=y_train)
feature_names = list(X.columns)
print(feature_names)

['MedInc', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'Over25_Over25', 'Over25_Under25']


In [16]:
xgb_model = xgb.train(
    params={"seed": 1, "max_depth": 6, "min_child_weight": 20}, dtrain=dtrain
)

### Model interpreter with ohe

In [17]:
single_model_contribution = ModelInterpreter(feature_names, one_hot_cols=["Over25"])

In [18]:
single_model_contribution.fit(xgb_model)

In [19]:
single_row = X_test[feature_names].head(1)
print(single_row)

       MedInc  AveRooms  AveBedrms  Population  AveOccup  Latitude  Longitude  \
20046  1.6812  4.192201   1.022284      1392.0  3.877437     36.06    -119.01   

       Over25_Over25  Over25_Under25  
20046              0               1  


In [20]:
contribution_list = single_model_contribution.transform(
    single_row, return_feature_values=True, return_type="dicts"
)
print(contribution_list)

[{'MedInc': (1.6812, -0.6546311)}, {'Latitude': (36.06, -0.31241444)}, {'AveOccup': (3.8774373259052926, -0.2624968)}, {'Longitude': (-119.01, -0.17611653)}, {'AveRooms': (4.192200557103064, 0.03493051)}, {'Over25': ('Under25', -0.01715860329568386)}, {'Population': (1392.0, -0.00079436717)}, {'AveBedrms': (1.0222841225626742, -7.1596354e-05)}]
